In [ ]:
import json

import requests
import boto3
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport 
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import T
from gremlin_python.statics import long 

In [ ]:
g = traversal().with_remote(DriverRemoteConnection(
    'ws://janusgraph:8182/gremlin','g',
    transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))
)

g.V().hasLabel("paper").count().next(), g.E().hasLabel("citation").count().next()

In [ ]:
JANUS_GRAPH_ENDPOINT_URL = "ws://janusgraph:8182/gremlin"
S3_ENDPOINT_URL = "http://s3:9000"
EXPORT_SERVICE_URL = "http://export_service:8081/export"
PROCESSING_SERVICE_URL = "http://processing_service:8082/processing"
TRAIN_SERVICE_URL = "http://train_service:8083/modeltraining"
ENDPOINT_SERVICE_URL = "http://endpoint_service:8084/call"

In [ ]:
cora_export = {
    "command": "export-pg",
    "output_s3_path": S3_ENDPOINT_URL,
    "bucket": "test-bucket",
    "params": {
        "endpoint": JANUS_GRAPH_ENDPOINT_URL
    },
    "additional_params": {
        "jobs": [
            {
                "name": "new_cora_job",
                "target": {
                    "node": "paper",
                    "property": "category",
                    "type": "classification",
                    "split_rate": [0.1, 0.2, 0.7],
                },
                "features" :[
                    {
                        "node": "paper",
                        "property": "word_vector",
                        "type": "float_array"
                    },
                    {
                        "edge": ["paper", "citation", "paper"]
                    }
                ]
            }
        ]
    }
}

requests.post(
    EXPORT_SERVICE_URL, data=json.dumps(cora_export)
)

In [ ]:
cora_proc = {
    "s3_endpoint_url": S3_ENDPOINT_URL,
    "bucket": "test-bucket",
    "job_path": "janusgraph_ml/cora_job",
    "processed_data_s3_location": "janusgraph_ml/cora_processed",
    "config_file_name": "train_config.json"
}

requests.post(
    PROCESSING_SERVICE_URL, data=json.dumps(cora_proc)
)

In [ ]:
cora_train = {
    "s3_params": {
      "bucket": "test-bucket",
      "s3_endpoint_url": S3_ENDPOINT_URL
    },
    "train_config_s3_key": "janusgraph_ml/cora_job/train_config.json",
    "processing_config_s3_key": "janusgraph_ml/cora_processed/processing_config.json" 
}

requests.post(TRAIN_SERVICE_URL, data=json.dumps(cora_train))

In [ ]:
g = traversal().with_remote(DriverRemoteConnection(
    JANUS_GRAPH_ENDPOINT_URL,'g',
    transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))
)

In [ ]:
nodes = g.V().hasLabel('paper').project('id', 'category').by(T.id).by('category').next(10)
nodes

In [ ]:
prediction = (
    g.call("predict")
        .with_("endpoint_id", "cora_job")
        .with_("predict_entity_idx", '788578368')
        .with_("interface", "inductive")
        .next()
)

prediction

In [ ]:
prediction = (
    g.call("predict")
        .with_("endpoint_id", "cora_job")
        .with_("predict_entity_idx", '788607040')
        .with_("interface", "transductive")
        .next()
)

prediction